In [1]:
import os
import re
import torch
import numpy as np
import pandas as pd
from datasets import load_dataset
from fastcore.parallel import parallel
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

In [2]:
torch.tensor(0).to(0), torch.tensor(0).to(1)

(tensor(0, device='cuda:0'), tensor(0, device='cuda:1'))

In [3]:
def convert_to_chat_input(question):

    system =  """You are an AI assistant, answering multiple choice questions. 
Only output the letter (A,B,C,D,E,etc..) of the answer and nothing else."""
    
    messages = [
        {"role": "system", "content":system},
        {"role": "user", "content": question},
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [4]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
NUM_GPUS = torch.cuda.device_count(); NUM_GPUS

2

In [6]:
# eval_ds = load_dataset("pharaouk/dharma-2")['dharma_g1i5_shuffled']
# set(eval_ds['output'])

In [7]:
eval_ds = load_dataset("MixEval/MixEval", 'MixEval')['free_form']

In [8]:
eval_ds[9]

{'id': '9',
 'problem_type': 'free-form',
 'context': 'None',
 'prompt': 'In the 1833 Factory Act in Britain what was the minimum age of a child allowed to work in a factory?',
 'target': ['Nine years'],
 'benchmark_name': 'TriviaQA',
 'options': ['None']}

In [9]:
eval_ds = load_dataset("MixEval/MixEval", 'MixEval')['multiple_choice']

In [10]:
eval_ds[7]

{'id': '7',
 'problem_type': 'multiple-choice',
 'context': 'None',
 'prompt': 'Which of the following drugs is classified as a stimulant?',
 'target': ['1'],
 'benchmark_name': 'MMLU',
 'options': ['alcohol', 'nicotine', 'heroin', 'phencyclidine']}

In [11]:
MULTIPLE_CHOICE_PROMPT = """{context}

{prompt}
{enumerated_options}

Answer:"""

In [12]:
num2letter = dict(enumerate("ABCDEFGH"))

In [13]:
def process_ds(example):
    if example['context'] == 'None':
        example['context'] = ''
    example["enumerated_options"] = "\n".join([f"{num2letter[idx]}) {opt}" 
                                           for idx, opt in enumerate(example['options'])])
    example["output"] = num2letter[int(example['target'][0])]
    example["input"] = MULTIPLE_CHOICE_PROMPT.format_map(example) 
    return example

In [14]:
eval_ds = eval_ds.map(process_ds)

In [15]:
print(eval_ds[14]['input'])



What is a common use of the mineral graphite?
A) to make glass
B) as a source of iron
C) to make pencil leads
D) as a household cleaner

Answer:


In [16]:
prompts = [convert_to_chat_input(t) for t in eval_ds['input']]

In [17]:
print(prompts[0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an AI assistant, answering multiple choice questions. 
Only output the letter (A,B,C,D,E,etc..) of the answer and nothing else.<|eot_id|><|start_header_id|>user<|end_header_id|>

The book announced an insane world of dehumanization through terror in which the individual was systematically obliterated by an all-powerful elite. Its key phrases‚ÄîBig Brother, doublethink, Newspeak, the Ministry of Peace (devoted to war), the Ministry of Truth (devoted to lies), the Ministry of Love (devoted to torture)‚Äîburned their way at once into the modern consciousness. The passage above discusses
A) E.M. Forster's A Passage to India
B) Thomas Pynchon's V.
C) George Orwell's 1984
D) Flannery O'Connor's The Violent Bear It Away

Answer:<|eot_id|><|start_header_id|>assistant<|end_header_id|>




### llama-3-8b-instruct

This is the baseline llama chat model.

Dharma-2 : `acc:0.58`

Mix-Eval (Multiple Choice): `acc:72`

In [ ]:
MODEL_NAME = "meta-llama/Meta-Llama-3-8B-Instruct"
llm = LLM(model=MODEL_NAME, tensor_parallel_size=2, dtype="bfloat16")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
2024-06-13 06:41:10,338	WARNING utils.py:580 -- Detecting docker specified CPUs. In previous versions of Ray, CPU detection in containers was incorrect. Please ensure that Ray has enough CPUs allocated. As a temporary workaround to revert to the prior behavior, set `RAY_USE_MULTIPROCESSING_CPU_COUNT=1` as an env var before starting Ray. Set the env var: `RAY_DISABLE_DOCKER_CPU_WARNING=1` to mute this warning.
2024-06-13 06:41:10,339	WARNING utils.py:592 -- Ray currently does not support initializing Ray with fractional cpus. Your num_cpus will be truncated from 16.15 to 16.
2024-06-13 06:41:10,476	INFO worker.py:1753 -- Started a local Ray instance.


INFO 06-13 06:41:11 config.py:623] Defaulting to use mp for distributed inference
INFO 06-13 06:41:11 llm_engine.py:161] Initializing an LLM engine (v0.5.0) with config: model='meta-llama/Meta-Llama-3-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=meta-llama/Meta-Llama-3-8B-Instruct)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


(VllmWorkerProcess pid=34718) INFO 06-13 06:41:15 multiproc_worker_utils.py:215] Worker ready; awaiting tasks
(VllmWorkerProcess pid=34718) ERROR 06-13 06:41:15 multiproc_worker_utils.py:226] Exception in worker VllmWorkerProcess while processing method init_device: CUDA error: invalid device ordinal
(VllmWorkerProcess pid=34718) ERROR 06-13 06:41:15 multiproc_worker_utils.py:226] CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
(VllmWorkerProcess pid=34718) ERROR 06-13 06:41:15 multiproc_worker_utils.py:226] For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
(VllmWorkerProcess pid=34718) ERROR 06-13 06:41:15 multiproc_worker_utils.py:226] Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
(VllmWorkerProcess pid=34718) ERROR 06-13 06:41:15 multiproc_worker_utils.py:226] , Traceback (most recent call last):
(VllmWorkerProcess pid=34718) ERROR 06-13 06:41:15 multiproc_worker_utils.py:226]   File "

In [137]:
outputs = llm.generate(prompts, SamplingParams(temperature=0.0, max_tokens=1, stop=["<|eot_id|>"]))

Processed prompts: 100%|██████████| 2000/2000 [00:45<00:00, 43.61it/s, Generation Speed: 43.61 toks/s]


In [138]:
preds = [o.outputs[0].text for o in outputs]

In [139]:
acc = np.mean([p==a for p,a in zip(preds, eval_ds['output'])])
acc

0.72

### llama-3-8b-instruct-hqq

This is the model with just HQQ quantization.

Dharma-2 : `acc:0.6`

Mix-Eval (Multiple Choice): `acc:0.6905`

In [15]:
model_dir = "/workspace/models/llama-3-8b-instruct-hqq-dora-plus-plus-only-hqq-vllm"
llm = LLM(model=model_dir, tokenizer="meta-llama/Meta-Llama-3-8B-Instruct", 
            dtype="bfloat16", tensor_parallel_size=NUM_GPUS, enforce_eager=False,
            quantization="torchao", gpu_memory_utilization=0.9)

WARNING 06-12 21:13:35 config.py:187] torchao quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 06-12 21:13:35 llm_engine.py:103] Initializing an LLM engine (v0.4.2) with config: model='/workspace/models/llama-3-8b-instruct-hqq-dora-plus-plus-only-hqq-vllm', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=torchao, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=/workspace/models/llama-3-8b-instruct-hqq-dora-plus-plus-only-hqq-vllm)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 06-12 21:13:37 selector.py:37] Using FlashAttention-2 backend.
INFO 06-12 21:13:40 model_runner.py:145] Loading model weights took 9.3189 GB
INFO 06-12 21:14:10 gpu_executor.py:83] # GPU blocks: 14326, # CPU blocks: 2048
INFO 06-12 21:14:11 model_runner.py:818] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 06-12 21:14:11 model_runner.py:822] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 06-12 21:15:23 model_runner.py:888] Graph capturing finished in 71 secs.


In [19]:
outputs = llm.generate(prompts, SamplingParams(temperature=0.0, max_tokens=1, stop=["<|eot_id|>"]))

Processed prompts: 100%|██████████| 2000/2000 [01:05<00:00, 30.39it/s, Generation Speed: 30.39 toks/s]


In [17]:
preds = [o.outputs[0].text for o in outputs]

In [18]:
acc = np.mean([p==a for p,a in zip(preds, eval_ds['output'])])
acc

0.6905

### llama-3-8b-instruct-hqq-dora

This is the model with HQQ quantization and HQQ++ dataset dora finetuning.

Dharma-2 : `acc:0.6`

Mix-Eval (Multiple Choice): `acc:0.7085`



In [ ]:
model_dir = "/workspace/models/llama-3-8b-instruct-hqq-dora-plus-plus-qdora-vllm/"
llm = LLM(model=model_dir, tokenizer="meta-llama/Meta-Llama-3-8B-Instruct", 
            dtype="bfloat16", tensor_parallel_size=NUM_GPUS, enforce_eager=False,
            quantization="torchao", gpu_memory_utilization=0.9)

WARNING 06-13 05:30:36 config.py:187] torchao quantization is not fully optimized yet. The speed can be slower than non-quantized models.


2024-06-13 05:30:38,930	WARNING utils.py:580 -- Detecting docker specified CPUs. In previous versions of Ray, CPU detection in containers was incorrect. Please ensure that Ray has enough CPUs allocated. As a temporary workaround to revert to the prior behavior, set `RAY_USE_MULTIPROCESSING_CPU_COUNT=1` as an env var before starting Ray. Set the env var: `RAY_DISABLE_DOCKER_CPU_WARNING=1` to mute this warning.
2024-06-13 05:30:38,934	WARNING utils.py:592 -- Ray currently does not support initializing Ray with fractional cpus. Your num_cpus will be truncated from 16.15 to 16.
2024-06-13 05:30:40,050	INFO worker.py:1753 -- Started a local Ray instance.


INFO 06-13 05:30:40 llm_engine.py:103] Initializing an LLM engine (v0.4.2) with config: model='/workspace/models/llama-3-8b-instruct-hqq-dora-plus-plus-qdora-vllm/', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, disable_custom_all_reduce=False, quantization=torchao, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=/workspace/models/llama-3-8b-instruct-hqq-dora-plus-plus-qdora-vllm/)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


(RayWorkerWrapper pid=10366) INFO 06-13 05:30:50 selector.py:37] Using FlashAttention-2 backend.
INFO 06-13 05:30:50 selector.py:37] Using FlashAttention-2 backend.
(RayWorkerWrapper pid=10366) ERROR 06-13 05:30:51 worker_base.py:145] Error executing method init_device. This might cause deadlock in distributed execution.
(RayWorkerWrapper pid=10366) ERROR 06-13 05:30:51 worker_base.py:145] Traceback (most recent call last):
(RayWorkerWrapper pid=10366) ERROR 06-13 05:30:51 worker_base.py:145]   File "/workspace/git/vllm_fork/vllm/worker/worker_base.py", line 137, in execute_method
(RayWorkerWrapper pid=10366) ERROR 06-13 05:30:51 worker_base.py:145]     return executor(*args, **kwargs)
(RayWorkerWrapper pid=10366) ERROR 06-13 05:30:51 worker_base.py:145]   File "/workspace/git/vllm_fork/vllm/worker/worker.py", line 105, in init_device
(RayWorkerWrapper pid=10366) ERROR 06-13 05:30:51 worker_base.py:145]     torch.cuda.set_device(self.device)
(RayWorkerWrapper pid=10366) ERROR 06-13 05:

2024-06-13 05:30:56,716	ERROR worker.py:406 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::RayWorkerWrapper.execute_method() (pid=10366, ip=192.168.226.2, actor_id=7f3cd4cb789611ca5a2e2c1101000000, repr=<vllm.executor.ray_utils.RayWorkerWrapper object at 0x7f0e84e65960>)
  File "/workspace/git/vllm_fork/vllm/worker/worker_base.py", line 146, in execute_method
    raise e
  File "/workspace/git/vllm_fork/vllm/worker/worker_base.py", line 137, in execute_method
    return executor(*args, **kwargs)
  File "/workspace/git/vllm_fork/vllm/worker/worker.py", line 105, in init_device
    torch.cuda.set_device(self.device)
  File "/usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py", line 399, in set_device
    torch._C._cuda_setDevice(device)
RuntimeError: CUDA error: invalid device ordinal
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKI

In [19]:
outputs = llm.generate(prompts, SamplingParams(temperature=0.0, max_tokens=1, stop=["<|eot_id|>"]))

Processed prompts: 100%|██████████| 2000/2000 [01:16<00:00, 25.98it/s, Generation Speed: 25.98 toks/s]


In [20]:
preds = [o.outputs[0].text for o in outputs]

In [21]:
acc = np.mean([p==a for p,a in zip(preds, eval_ds['output'])])
acc

0.7085

In [26]:
def create_chat_text(tokenizer, user_prompt, system="You are a helpful assistant."):
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": user_prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    return messages, text

### benchmark

In [81]:
import time

In [46]:
INPUT_TOKENS = 4000

In [47]:
reps = "hello world!\n" * 8000
dummy_input = f"Repeat this indefinitely: {reps}"
_,long_prompt = create_chat_text(tokenizer, dummy_input)
truncated_tokens = tokenizer.encode(long_prompt, add_special_tokens=False)[:INPUT_TOKENS-1]
prompt = tokenizer.decode(truncated_tokens)
assert len(tokenizer.encode(prompt)) == INPUT_TOKENS
len(tokenizer.encode(prompt))

4000

In [ ]:
llm.llm_engine.model_executor.

In [119]:
2048*3

6144

In [121]:
for BS in [1, 4, 8, 16]:
    for INPUT_TOKENS in [1024, 2048, 4096, 6144]:
        for MAX_TOKENS in [32, 64, 128, 256, 512, 1024, 2048, 4096]:
            if INPUT_TOKENS + MAX_TOKENS < 8192:
                print(INPUT_TOKENS, MAX_TOKENS)

1024 32
1024 64
1024 128
1024 256
1024 512
1024 1024
1024 2048
1024 4096
2048 32
2048 64
2048 128
2048 256
2048 512
2048 1024
2048 2048
2048 4096
4096 32
4096 64
4096 128
4096 256
4096 512
4096 1024
4096 2048
6144 32
6144 64
6144 128
6144 256
6144 512
6144 1024
1024 32
1024 64
1024 128
1024 256
1024 512
1024 1024
1024 2048
1024 4096
2048 32
2048 64
2048 128
2048 256
2048 512
2048 1024
2048 2048
2048 4096
4096 32
4096 64
4096 128
4096 256
4096 512
4096 1024
4096 2048
6144 32
6144 64
6144 128
6144 256
6144 512
6144 1024
1024 32
1024 64
1024 128
1024 256
1024 512
1024 1024
1024 2048
1024 4096
2048 32
2048 64
2048 128
2048 256
2048 512
2048 1024
2048 2048
2048 4096
4096 32
4096 64
4096 128
4096 256
4096 512
4096 1024
4096 2048
6144 32
6144 64
6144 128
6144 256
6144 512
6144 1024
1024 32
1024 64
1024 128
1024 256
1024 512
1024 1024
1024 2048
1024 4096
2048 32
2048 64
2048 128
2048 256
2048 512
2048 1024
2048 2048
2048 4096
4096 32
4096 64
4096 128
4096 256
4096 512
4096 1024
4096 2048
6144 

In [105]:
BS = 16

In [106]:
MAX_TOKENS = 1024

In [113]:
s = time.time()
output = llm.generate([prompt]*BS, SamplingParams(temperature=0.0, max_tokens=1))
e = time.time()

Processed prompts: 100%|██████████| 16/16 [00:15<00:00,  1.06it/s, Generation Speed: 1.06 toks/s]


In [114]:
ttft = e - s; ttft

15.223310470581055

In [115]:
s = time.time()
output = llm.generate([prompt]*BS, SamplingParams(temperature=0.0, max_tokens=MAX_TOKENS))
e = time.time()

Processed prompts: 100%|██████████| 16/16 [00:58<00:00,  3.63s/it, Generation Speed: 281.83 toks/s]


In [116]:
completion_time = e - s; completion_time

58.26063251495361

In [117]:
lat_tok_per_sec = MAX_TOKENS / (completion_time - ttft); tok_per_sec

23.793301984362092

In [118]:
tput_tok_per_sec = BS*MAX_TOKENS / (completion_time - ttft); tok_per_sec

380.69283174979347

In [3]:
baseline_df = pd.read_csv("/workspace/git/kerem_research/inference_benchmarking/baseline.csv")
qdora_df    = pd.read_csv("/workspace/git/kerem_research/inference_benchmarking/qdora.csv")

In [5]:
baseline_df.shape, qdora_df.shape

((84, 7), (84, 7))

In [11]:
bench_df = baseline_df.merge(qdora_df, on=["BS", "INPUT_TOKENS", "MAX_TOKENS"], how="inner", suffixes=["_baseline", "_qdora"]); bench_df.head()

,BS,INPUT_TOKENS,MAX_TOKENS,time_to_first_token_baseline,completion_time_baseline,lat_tok_per_sec_baseline,tput_tok_per_sec_baseline,time_to_first_token_qdora,completion_time_qdora,lat_tok_per_sec_qdora,tput_tok_per_sec_qdora
0,1,1024,128,0.150467,3.934720,33.824378,33.824378,0.281821,1.981092,75.326379,75.326379
1,1,1024,256,0.150341,7.747869,33.695169,33.695169,0.282049,3.690753,75.101866,75.101866
2,1,1024,512,0.150956,15.378392,33.623521,33.623521,0.282219,7.106683,75.024201,75.024201
3,1,1024,1024,0.152311,30.735919,33.481988,33.481988,0.283652,14.015289,74.572318,74.572318
4,1,1024,2048,0.151235,61.438495,33.416406,33.416406,0.283724,27.714202,74.661475,74.661475


In [13]:
bench_df.query("lat_tok_per_sec_qdora < lat_tok_per_sec_baseline")

,BS,INPUT_TOKENS,MAX_TOKENS,time_to_first_token_baseline,completion_time_baseline,lat_tok_per_sec_baseline,tput_tok_per_sec_baseline,time_to_first_token_qdora,completion_time_qdora,lat_tok_per_sec_qdora,tput_tok_per_sec_qdora


In [15]:
bench_df["improvement"] = bench_df["lat_tok_per_sec_qdora"] / bench_df["lat_tok_per_sec_baseline"]

In [18]:
bench_df = bench_df.sort_values("improvement", ascending=False)

In [20]:
pd.options.display.max_rows = 100

In [23]:
np.median(bench_df["improvement"]), np.mean(bench_df["improvement"])

(1.7897560673956545, 1.7478800401891452)

In [21]:
bench_df

,BS,INPUT_TOKENS,MAX_TOKENS,time_to_first_token_baseline,completion_time_baseline,lat_tok_per_sec_baseline,tput_tok_per_sec_baseline,time_to_first_token_qdora,completion_time_qdora,lat_tok_per_sec_qdora,tput_tok_per_sec_qdora,improvement
4,1,1024,2048,0.151235,61.438495,33.416406,33.416406,0.283724,27.714202,74.661475,74.661475,2.234276
2,1,1024,512,0.150956,15.378392,33.623521,33.623521,0.282219,7.106683,75.024201,75.024201,2.231301
1,1,1024,256,0.150341,7.747869,33.695169,33.695169,0.282049,3.690753,75.101866,75.101866,2.228862
8,1,2048,512,0.294153,15.557348,33.544748,33.544748,0.532325,7.384617,74.719524,74.719524,2.227458
3,1,1024,1024,0.152311,30.735919,33.481988,33.481988,0.283652,14.015289,74.572318,74.572318,2.227237
0,1,1024,128,0.150467,3.934720,33.824378,33.824378,0.281821,1.981092,75.326379,75.326379,2.226985
7,1,2048,256,0.294467,7.909505,33.617691,33.617691,0.533354,3.953269,74.855647,74.855647,2.226674
6,1,2048,128,0.296854,4.089521,33.749343,33.749343,0.532804,2.237246,75.097869,75.097869,2.225165
9,1,2048,1024,0.295098,30.906995,33.451047,33.451047,0.532820,14.298145,74.389817,74.389817,2.223841
5,1,1024,4096,0.155924,123.292990,33.263745,33.263745,0.283646,55.872633,73.683659,73.683659,2.215134
